Author: Nick Havlicek (nhavlicek@wisc.edu) 
Last edit: May 23, 2020

Adapted from MATLAB code written by Brandon Wilson for the purpose of altitude prediction given a set of physical parameters. This has a similar print results, but is focused on providing optimized estimates for tank sizing.

If you have MATLAB and are familiar with it, I would suggest reading his original code first, as I made some changes specific to Julia to make it do what I wanted it to do. 
If you dont have MATLAB installed and would still perfer to read the MATLAB code, the html folder inside the alt_prediction 3 folder is where you should go.

The original MATLAB code dependes on trial and error inputs to choose suboptimal, but realistic, tank dimensions. This rewrite looks to optimize these suboptimal dimensions, as well as performing sensitivity analysis on parameters relevant to rocket development.

# Introduction

This notebook alters preexisting MATLAB code originally written to calculate a rough estimate of predicted altitude given certain rocket dimensions and parameters. The code was originally written by Brandom Wilson and is included in the folder.

### Problem to be solved

For a single stage rocket to reach the highest altitude it can, one thing a rocket can do is produce thrust for the longest duration possible. These is primarily achieved by maximizing the amount of propellant in the rocket. Two separate tanks of Oxygen and Kerosene (https://en.wikipedia.org/wiki/RP-1) must hold an optimal amount of fuel such that they both run out at the same time. For this to be accomplished, the flow of fuel through the engine must be calculated or measured precicely, and the tanks must be sized accordingly. A third tank of pressurant will feed through the top of both tanks to keep propellant mass flow constant and as high as possible.

Feasibility of the model was considered with the help of Lucy Fitzgerald and Finn Roberts, the propulsion team lead and the structures team lead respectively.

#### The variables

An existing small rocket skeleton already exists, however exactly how the tanks will fit has not been decided. The chamber size for the tanks can be measured. Other than that, there is some flexibility of the design that will be up to Structures.

There are further constraints that may be accounted for which are not in the code. This is a future problem for this project.

##### Things to consider in the future

- Centrifugal force on propellant due to roll of the rocket in flight
- Expansion of tanks due to temperature changes. 
- Excess Fuel optimality. Which one should run out first?

#### The parameters

Rocket parameters are either obtained from the physically existing small-scale rocket or are roughly estimated. Since this rocket is currently being built, a lot of the data is subject to change. This code will work for the large-scale rocket as far as I am aware.

#### Notes on the models

The important thing to note is that mixture ratio, chamber pressure, combustion performance, and chamber temperature are all theoretically choosen based on the chemistry of the fuel. Some of these will be determined as we test the engine as it is being built.

# Models

### Model 1: tank
#### Variables
    Amount of propellant: mass of liquid oxygen + mass of Kerosene (https://en.wikipedia.org/wiki/RP-1)
    Amount of pressurant: mass of Nitrogen
#### Constraint
    Tank length: length of the Oxygen tank + length of the Kerosene tank + length of Pressurant tank
    Tank diameter: existing rocket dimension
    Fuel ratio: 2.56 parts Oxygen to 1 part Kerosene
    Pressurant: Enough pressurant to maintain steady fuel flow for the duration of the burn
#### Objective
    Maximize amount of propellant
    
This model is capable of returning optimum tank dimensions for specific rocket parameters. Any changes in rocket design only need to be made in the definitions of the specifications in the code in order for this model to return a corrected measuremet. This model is the main feature that will be used immediatly in the design process of the internal structure of the rocket. 

### Model 2 & 3: OxygenTankPressure, FuelTankPressure
#### Variables
    mass flow of oxidizer: the amount of the oxidizer that flows through the engine per time
    mass flow of Kerosene: the amount of Kerosene that flows through the engine per time
    ox:fuel ratio: ratio of mass flows of the propellant. Since it's the flow rate that matters, the tanks do not share 
        this ratio in an obvious manner
    
#### Constraint
    Constraints based on fluid dynamics, and thus is formally explained in the Propulsion Theory power point
#### Objective
    Maximize tank pressure
    
This model is mostly to demonstrate the capability and functionality of sensitivity analysis within the code, and not because these specific variables are the most important to rocket design. As engine design moves further along, sensitivity will be performed at the direction of team leads.

# Code
Without using extensions, the code must be organized in a difficult to follow manner. It is written so that a simple "run all cells" will compile correctly. Each block has a description of what it does, and most are function definitions. If they don't immediatly make sense, follow each function call starting near the bottom. It is a mess, but without jupyter notebook extensions there is hardly a better way.

If you followed the installation instructions correctly, running this block will check for package updates or it will install them if you havent already.

In [111]:
using Pkg
Pkg.add("JuMP")
Pkg.add("LinearAlgebra")
Pkg.add("DifferentialEquations")
Pkg.add("Gurobi")
Pkg.add("Ipopt")
Pkg.add("PyPlot")
using JuMP, LinearAlgebra, DifferentialEquations, Gurobi, Ipopt, PyPlot

 Resolving package versions...
  Updating `C:\Users\Nick\.julia\environments\v1.3\Project.toml`
 [no changes]
  Updating `C:\Users\Nick\.julia\environments\v1.3\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Nick\.julia\environments\v1.3\Project.toml`
 [no changes]
  Updating `C:\Users\Nick\.julia\environments\v1.3\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Nick\.julia\environments\v1.3\Project.toml`
 [no changes]
  Updating `C:\Users\Nick\.julia\environments\v1.3\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Nick\.julia\environments\v1.3\Project.toml`
 [no changes]
  Updating `C:\Users\Nick\.julia\environments\v1.3\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\Users\Nick\.julia\environments\v1.3\Project.toml`
 [no changes]
  Updating `C:\Users\Nick\.julia\environments\v1.3\Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `C:\User

############## Universal Constants ###################

In [83]:
g = 9.807;               # gravitational constant [m/s^2]
R_bar = 8314.3;          # universal gas constant [J/kmol-K]
kg2lbm = 2.20462;        # kg to lbm conversion
lbm2kg = 0.453592;       # lbm to kg conversion
Pa2psi = 0.000145038;    # Pascal to psi conversion
psi2Pa = 6894.76;        # psi to Pascal conversion
m2ft = 3.28084;          # meter to foot conversion
ft2m = 0.3048;           # foot to meter conversion
in2m = 0.0254;           # inch to meter conversion
kN2lbf = 224.81;         # kN to lbf conversion

newtzero: This function was written by Matt Fig in MATLAB and has been modified for Julia. It is called in GetMassFlowAndNozzleDimensions().

In [84]:
function newtzero(f, xr)
    mx = 100 # max iterations
    tol = 1e-13 # tolerance for convergance 

# NEWTZERO finds roots of function using unique approach to Newton's Method.
# May find more than one root, even if guess is way off.  The function f
# should be an inline object or function handle AND capable of taking
# vector arguments.  The user can also pass in a string function.
# NEWTZERO takes four arguments. The first argument is the function to be
# evaluated, the second is an optional initial guess, the third is an
# optional number of iterations, and the fourth is an absolute tolerance.
# If the optional arguments are not supplied, the initial guess will be set
# to 1, the number of iterations will be set to 30, and the tolerance will
# be set to 1e-13.  If the initial guess is a root, no iterations will take
# place.
#
#
# Author:  Matt Fig
# Contact:  popkenai@yahoo.com

    if abs(f(xr))< tol
        global root = xr; # The user supplied a root as the initial guess.
        return  # Initial guess correct.
    end
    
    # apparently log ranges are harder than could ever be imagined in julia https://github.com/JuliaLang/julia/pull/25896
        #~NH~ 
    logrange(x1, x2, n) = (10^y for y in range(log10(x1), log10(x2), length=n))
    density = 220
    LGS = collect(logrange(0.00001,3,density)); # Can be altered for wider range or denser search.

    startvar = 0
    stopvar = 18/19
    LNS = range(startvar, stop=(stopvar), step=((stopvar-startvar)/(density-1))); # Search very close to initial guess too
        #sorry this is so brutal but matlab somehow handles line 81 if the 4 columns are different lengths, idk how.
    xrLGS = xr.*(ones(length(LGS),1))    
    xrLNS = xr.*(ones(length(LNS),1))    
    xr = transpose([xr.-LGS xr.+LGS xr.-LNS xr.+LNS]);  # Make vector of guesses.
    iter = 1;  # Initialize the counter for the while loop.
    mn1 = .1; # These will store the norms of the converging roots.
    mn2 = 1; # See last comment.
    sqrteps = sqrt(eps(Float64)); # Used to make h.  See loop.

    while iter <= mx && abs(mn2-mn1) > 5*eps(Float64)
        h = sqrteps.*xr; # From numerical recipes, make h = h(xr)
        xr = @. xr.-f(xr)./((f(xr.+h).-f(xr.-h))./(2*h)); # Newton's method.
        xr[isnan.(xr)] = []; # No need for these anymore.
        xr[isinf.(xr)] = []; # No need for these anymore.
        mn1 = mn2; # Store the old value first.
        mn2 = norm(xr); # This could make the loop terminate early!
        iter = iter+1;  # Increment the counter.
    end

    
    if abs(f(0)) < tol # The above method will tend to send zero root to Inf.
        xr = [xr;0];  # So explicitly check.
    end

    # Filtering.  We want to filter out certain common results.
    idxi = abs.(imag.(xr)) .< 5e-15;  # A very small imag term is zero.
    xr[idxi] = real.(xr[idxi]);  # Discard small imaginary terms.
    idxr = abs.(real.(xr)) .< 5e-15;  # A very small real term is zero.
    xr[idxr] = complex.(0,imag.(xr[idxr])); # Discard small real terms.
    root = xr[abs.(f.(xr)) .< tol]; # Apply the tolerance.

    # Next we are going to delete repeat roots.  unique does not work in
    # this case because many repeats are very close to each other but not
    # equal.  For loops are fast enough here, most root vectors are short(ish).

    if ~isempty(root)
        cnt = 1;  # Counter for while loop.
        rt = []
        while ~isempty(root)
            vct = abs.(root .- root[1]).<5e-6; # Minimum spacing between roots.
            C = root[vct];  # C has roots grouped close together.
            idx = argmin(abs.(f.(C)));  # Pick the best root per group.
            append!(rt,C[idx]);  # #ok<AGROW>  Most root vectors are small.
            root = root[.~vct]; # Deplete the pool of roots.
            cnt = cnt + 1;  # Increment the counter.
        end
    root = transpose(sort(rt));  # return a nice, sorted column vector
    end
    
    return root      
end
;

GetMassFlowAndNozzleDimensions Function
returns mass flow based on initial thrust

In [85]:
function GetMassFlowAndNozzleDimensions(F_T_0)

    epsilon = expansionRatio;
    P0 = P0_eng;
    gamma = gamma_prop;
    T0 = T0_eng;
    R = R_bar/M_bar_prop;
    P_atm = 101.3e3; # atmospheric pressure [Pascals]

    # calculate Me from expansion ratio. See pg 28 of Propulsion Theory pp
    Ae_Astar_expr = Me -> 1/Me.*(2/(gamma+1).*(1+(gamma-1)./2*Me.^2)).^((gamma+1)./(2*(gamma-1))) - epsilon;
    global Me = newtzero(Ae_Astar_expr, 1);

    # 2 solutions, only want supersonic
    if(length(Me) == 2)
        Me = Me[2];
    end
    # get exit pressure and temp
    Pe = P0*(1+(gamma-1)/2*Me^2)^(gamma/(1-gamma));
    global Pe_eng = Pe;
    Te = T0*(1+(gamma-1)/2*Me^2)^(-1);

    # solving for throat area and m_dot
    rho0 = P0/(R*T0);
    rho_t = rho0*(1+(gamma-1)/2)^(-1/(gamma-1));
    Tt = T0*(1+(gamma-1)/2)^(-1);
    At = F_T_0/((Pe-P_atm)*epsilon+Me*rho_t*gamma*R*sqrt(Tt*Te));
    global m_dot = rho_t*At*sqrt(gamma*R*Tt);

    # exit area, velocity and ISP
    global Ae = epsilon*At;
    ue = Me*sqrt(gamma*R*Te);
    global Isp = ue/9.807;
    
    return Pe_eng, ue, m_dot    
end
;

Get_rho_T_P Function
returns temperature, pressure and density from an altitude input

In [86]:
function Get_rho_T_P(alt)
    # Model based on NASA.gov:
    # https://www.grc.nasa.gov/www/k-12/airplane/atmosmet.html

    #atmospheric definitions
    Tropopause = 11000;     # Altitude of initial Tropopause reg [m]
    lowerStrat = 25000;     # Altitude of lower stratosphere [m]

    # Temp linearly decreases then becomes constant at tropopause
    if(alt > lowerStrat)
        T = -131.21 + 0.00299*alt; #C
        P = 2.488*((T+273.15)/216.6)^(-11.388); # kPa
    elseif(alt > Tropopause && alt < lowerStrat)
        T = -56.46; # C
        P = 22.65*exp(1.73-0.000157*alt); #kPa
    else
        T = 15.04 - 0.00649*alt;    # C
        P = 101.29*((T+273.15)/288.08)^5.256; # kPa
    end

    # Air density as a function of pressure and temp [kg /m^3]
    rho = P/(0.2869*(T+273.15));

    # convert back to SI
    P = P*1000; #Pa
    T = T + 273.15; # K

    return rho,T,P
end
;

Get_flow Function
returns drag coefficient, Mach and Reynolds #

In [87]:
function GetFlow(T,rho,V)

    gamma = 1.4;            # Ratio of specific heat for air
    M_bar_air = 28.97;      # kg/kmol
    R = R_bar/M_bar_air;    # Ideal gas constant [j/(Kg-K)]
    c = sqrt(gamma*R*T);    # Speed of sound
    Ma = norm(V/c);         # Mach Number
    mu0 = 18.27e-6;         # Dynamic viscosity at 291.15 K [Pa-s]
    T0 = 291.15;
    C = 120;                # Sutherland constant
    mu = mu0*((T0+C)/(T+C))*(T/T0)^(3/2);   # Dynamic viscosity (Sutherland's eq)
    Re = rho*V*r_veh*2/mu; # Reynold's number

    # approximationg drag coefficient from "rocket ans spacecraft propulsion"
    # from Marting Turner (page 150). Esssentiall Cd is roughly constant in
    # subsonic regime, then exponentially increases to mach 1, then
    # exponentiall decreases.
    a = 0.15;
    b = 0.35;
    if(Ma < 1)
        Cd = a + b*Ma^6;
    else
        Cd = a + b/Ma^2;
    end

    return Cd, Ma, Re
end
;

GetFeedPress Function
returns tank pressure based on pressure drops, losses, and line diam and mass flow, density, and line length [m]

In [88]:
function GetFeedPress(lineDiam, P0_eng, catBed_pressDrop, inj_PressDrop, m_dot, rho, L_in, model)
    
    A = pi/4*lineDiam^2; # area [m^2]
    
    @variable(model, v)
    @NLconstraint(model, v == m_dot/(rho*A)) # velocity [m/s] 
    mu = 1.003e-3;  # viscosity (using water)
    @variable(model, Re >= 2200) # non-laminar flow if >= 2200
    @NLconstraint(model, Re == rho*v*lineDiam/mu); # reynolds number
    epsilon = 0.005/1000; # guess for pipe roughness
    L= L_in*in2m; # line length [m]

    @variable(model, x >= 0)
    @NLconstraint(model, -2*log(epsilon/lineDiam/3.7+2.51/(Re*sqrt(x)))-1/sqrt(x) == 0) # some fluid dynamic equation
    # Maximum tank_press. This can be changed to perform sensitivity analysis on different things in this function.
    @NLobjective(model, Max, 2*Pa2psi*(( rho*v^2 ) 
            + ( P0_eng+catBed_pressDrop+inj_PressDrop )
            + ( rho*x*L/lineDiam*v^2/2)) )
    optimize!(model)
    
    println("obj_val max tank pressure: " ,objective_value(model))
    f = value(x)
    v = value(v)
    deltaP = rho*f*L/lineDiam*v^2/2; 
    
    backPress = P0_eng+catBed_pressDrop+inj_PressDrop; # pressure at inlet
    return tank_press = rho*v^2 + backPress + deltaP; # pressure

end
;

GetPropTankVol Function
Returns tank volumes

In [89]:
function GetPropTankVol(OF, propellantMass,rho_ox, rho_f)
    
    m_ox = (OF * propellantMass) / (1 + OF);  # mass oxidizer [kg]
    m_f = propellantMass - m_ox;              # mass fuel [kg]
    vol_ox = m_ox / rho_ox;                   # Volume of oxidizer [m^3]
    vol_f = m_f / rho_f;                      # Volume of fuel [m^3]
    vol_ox_boilOff = vol_ox * 0.01;           # boil off volume (rough approx)
    vol_f_boilOff = vol_f * 0;                # non cryo, wont boil off
    vol_ox_ullage = (vol_ox_boilOff + vol_ox) * 0.05;
    vol_f_ullage = (vol_f_boilOff + vol_f) * 0.05;

    vol_ox_total = vol_ox + vol_ox_boilOff + vol_ox_ullage;
    vol_f_total = vol_f + vol_f_boilOff + vol_f_ullage;
    
    
    return vol_ox_total,vol_f_total,m_ox,m_f
end
;

propTankWt_Size function
returns overall tank weight, overall length, thickness of cylinder, thickness of hemispherical ends, thickness of junction, and length of tank

In [90]:
function propTankWt_Size(vol_total, r_tank, operatingPressure)
    
    # Parameters are:  outer radius of tank [ft], and operating pressure [psi]
    # Tank size and weight 

    rho_Al = 170;                       # Density of 6066 t-6 aluminum, lbm/ft^3
    al_yield = 39000;                   # Yield strength of 6066 t-6 al
    al_ultimate = 45000;                # Ultimate strength of 6066 t-6 al
    K = 1/0.67;                         # Knuckle factor for stress concentrations

    # Max allowable stress per MIL-STD1522A (USAF)
    maxAllowStress = min(al_yield/1.25, al_ultimate/1.5);
    
    ##############################################################################
    # Tank Thickness Calcs
    # Required wall thickness at the weld [in]
    
    t_k = K * operatingPressure * r_tank * 12 / maxAllowStress;

    # Required wall thickness of the hemispheircal ends [in]
    t_cr = operatingPressure * r_tank * 12 / (2 * maxAllowStress);

    # Hemispherical end thickness [in]
    t_hemi = (t_k + t_cr) / 2;

    # Required thickness of cylindrical section [in]
    t_cyl = operatingPressure * r_tank * 12 / maxAllowStress;
    
    ##############################################################################
    # Tank Sizing Calcs
    # Inner radii of hemi and cyl [ft]
    
    r_inner = r_tank - t_hemi/12;

    # volume of end cap
    vol_hemi = 2/3 * pi * r_inner^3;

    # Volume required for cylinders
    vol_cyl = vol_total - 2 * vol_hemi;

    # oxider tank length cylinder only
    l_tank_cyl = vol_cyl / (pi * r_inner^2);

    # tank overall Length (inner dimensions)
    tank_length = l_tank_cyl + 2 * r_inner;
    
    ##############################################################################
    # Weight Calcs
    # Weight of components
    
    weight_tank_cyl = rho_Al * pi * l_tank_cyl * (r_tank^2 - r_inner^2);
    weight_Tank_Hemis = 2/3 * pi * rho_Al * (r_tank^3 - r_inner^3);

    # Overall weight
    tankWt = weight_tank_cyl + 2 * weight_Tank_Hemis;
    
    
    return tankWt,tank_length,t_cyl,t_hemi,l_tank_cyl
end
;

Rocket Altitude Predictions
main function that calls sub functions and does plotting

In [91]:
function altitude_prediction(sim_t,R,r_tank,l_tank,m_struct,m_propellant,
        F_T,epsilon,P0,T0,M_bar,gamma,inj_percentDrop,catBed_pressDrop, 
        ox_lineDiameter,f_lineDiameter,ox_fuel_ratio)
   
############## Defining variables from input ###############
    global m_veh_structure = m_struct;
    global simTime = sim_t;
    global m_prop = m_propellant;
    global P0_eng = P0;
    global T0_eng = T0;
    global M_bar_prop = M_bar;
    global F_T_max = F_T;
    global gamma_prop = gamma;
    global expansionRatio = epsilon;
    global r_veh = R;

############# Initial Calculations ##################
    
    # mass flow, throat area, and exit area for engine
    massFlowAndNozzleArray = GetMassFlowAndNozzleDimensions(F_T_max) 
    global Pe_eng = massFlowAndNozzleArray[1];
    global ue     = massFlowAndNozzleArray[2];
    global m_dot  = massFlowAndNozzleArray[3];

    global eng_time = m_prop/m_dot;  # burn time
    global Pe = Pe_eng

    # Propellant tank calculations
    global rho_ox = 1141; # oxidizer density [kg/m^3]
    global rho_f = 820;   # fuel density [kg/m^3]
    global OF = ox_fuel_ratio; # ox:fuel ratio    
    
    global inj_PressDrop = inj_percentDrop/100*P0_eng; # pressure drop across injector

    
    ############# Sensitivity analysis Model 2 & 3 ####################
    
    OxygenTankPressure = Model(with_optimizer(Ipopt.Optimizer, print_level=0))
    @variable(OxygenTankPressure, OFVarOx)
    @constraint(OxygenTankPressure, OxRatioCon, OFVarOx == OF)
    @variable(OxygenTankPressure, m_dot_ox_Var)
    @NLconstraint(OxygenTankPressure, OxVelCon, m_dot_ox_Var == (OFVarOx * m_dot) / (1 + OFVarOx))
    global ox_lineLength = 24; # guess for ox run line length [in]
    
    # oxidizer tank pressure
    println("\nOxygen Tank")
    global ox_tankPressure = 2*GetFeedPress(ox_lineDiameter*in2m,P0_eng, catBed_pressDrop,inj_PressDrop,m_dot_ox_Var,
    rho_ox,ox_lineLength,OxygenTankPressure)
    
    println("dual m_dot_ox: ", dual(OxVelCon))
    global m_dot_ox = value(m_dot_ox_Var)
    println("mean ox_tankPressure ", 1/2*ox_tankPressure*Pa2psi)
    println()
    
    ############
    
    FuelTankPressure = Model(with_optimizer(Ipopt.Optimizer, print_level =0))
    @variable(FuelTankPressure, OFVarF)
    @constraint(FuelTankPressure, FuelRatioCon, OFVarF == OF)
    @variable(FuelTankPressure, m_dot_f_Var)
    @NLconstraint(FuelTankPressure, FVelCon, m_dot_f_Var == (m_dot - ((OFVarF * m_dot) / (1 + OFVarF))) )
    global f_lineLength = 144; # guess for fuel run line length [in]
    
    # fuel tank pressure
    println("Fuel Tank")
    global f_tankPressure = 2*GetFeedPress(f_lineDiameter*in2m, P0_eng, catBed_pressDrop*0, inj_PressDrop, m_dot_f_Var,
        rho_f, f_lineLength, FuelTankPressure)
    global m_dot_f = value(m_dot_f_Var)
    
    println("dual m_dot_f: ", dual(FVelCon))
    println("mean f_tankPressure ", 1/2*f_tankPressure*Pa2psi)
    println()
    
    ################

    # get tank volumes
    volArray = GetPropTankVol(OF,m_prop,rho_ox,rho_f)
    global vol_ox = volArray[1];
    global vol_f =  volArray[2];
    global m_ox=    volArray[3];
    global m_f =    volArray[4];

    # get ox tank size and weight estimates
    oxArray = propTankWt_Size(vol_ox*m2ft^3,r_tank,ox_tankPressure*Pa2psi)
    global ox_tankWt=        oxArray[1];
    global ox_tank_length =  oxArray[2];
    global ox_t_cyl =        oxArray[3];
    global ox_t_hemi=        oxArray[4];
    global ox_l_tank_cyl =   oxArray[5];
    # get fuel tank size and weight estimates
    fuelArray = propTankWt_Size(vol_f*m2ft^3,r_tank,f_tankPressure*Pa2psi)
    global f_tankWt =        fuelArray[1];
    global f_tank_length =   fuelArray[2];
    global f_t_cyl =         fuelArray[3];
    global f_t_hemi =        fuelArray[4];
    global f_l_tank_cyl =    fuelArray[5];
    
    @constraint(tank, ox_tank_length + f_tank_length <= l_tank)
    @objective(tank, Max, m_prop) # maximize propellant to maximize altitude.
    optimize!(tank) 
    global m_prop = objective_value(tank)
    
######### Redefine everything with m_prop decided #######
    
    global eng_time = m_prop/m_dot;  # burn time
    global Pe = Pe_eng
    
    # get tank volumes
    volArray = GetPropTankVol(OF,m_prop,rho_ox,rho_f)
    global vol_ox = volArray[1];
    global vol_f =  volArray[2];
    global m_ox=    volArray[3];
    global m_f =    volArray[4];

    # get ox tank size and weight estimates
    oxArray = propTankWt_Size(vol_ox*m2ft^3,r_tank,ox_tankPressure*Pa2psi)
    global ox_tankWt=        oxArray[1];
    global ox_tank_length =  oxArray[2];
    global ox_t_cyl =        oxArray[3];
    global ox_t_hemi=        oxArray[4];
    global ox_l_tank_cyl =   oxArray[5];
    # get fuel tank size and weight estimates
    fuelArray = propTankWt_Size(vol_f*m2ft^3,r_tank,f_tankPressure*Pa2psi)
    global f_tankWt =        fuelArray[1];
    global f_tank_length =   fuelArray[2];
    global f_t_cyl =         fuelArray[3];
    global f_t_hemi =        fuelArray[4];
    global f_l_tank_cyl =    fuelArray[5];

###############################################################    
    
    
    global tankWt = ox_tankWt + f_tankWt;

    global m_tanks = tankWt*lbm2kg; # propellant tank total

    global m_veh_empty = m_veh_structure+m_tanks;  # empty mass of vehicle [kg]
    global m_veh_wet = m_veh_empty+m_prop;         # wet mass of vehicle [kg]
    global A_veh = pi*r_veh^2;                     # Cross sectional area of vehicle [m^2]
end
;

In [92]:
tank = Model(with_optimizer(Gurobi.Optimizer, OutputFlag = 0))

m_veh_structure = 40; # dry mass of rocket [kg]


#m_prop = 20;      # Initial propellant mass, kg
@variable(tank, m_prop)
r_veh = 4/12*ft2m; # radius of vehicle [m]
epsilon = 4.5;     # expansion ratio of nozzle
F_T_max = 4000;    # Thrust when exit pressure = atm pressure [Newtons]
r_tank = 3/12;     # tank radius [ft]
l_tank = 4;         # length of the tank chamber [ft]

# feed pressure parameters
inj_percentDrop = 20;           # pressure drop across injector [% of chamber press]
catBed_pressDrop = 0*psi2Pa;    # pressure drop across catalyst bed [Pa]
ox_lineDiameter = 0.75-2*0.083; # ox feed line diameter [in]
f_lineDiameter = 0.5-2*0.035;   # fuel feed line diameter [in]

# from NASA Chemical Equilibrium with Applications
# The NASA Computer program CEA (Chemical Equilibrium with Applications) calculates chemical 
# equilibrium compositions and properties of complex mixtures.
P0_eng = 2.06e6;     # Initial pressure of engine [Pascals]
T0_eng = 3141;       # Initial temperature of engine [Kelvin]
M_bar_prop = 22.15;  # Molar mass of propellant [kg/kmol]
gamma_prop = 1.14;   # specific heat ratio of propellants (frozen-flow)

rho_ox = 1141;  # oxidizer density [kg/m^3]
rho_f = 820;    # fuel density [kg/m^3]
OF = 2.56;      # ox:fuel ratio
# according to wikipedia, RP-1 (https://en.wikipedia.org/wiki/RP-1) has an optimal ratio of 2.56.
# change this ratio with measured value of engine performance.


# call altitude prediction model
# calculates variables for the equations of motion
altitude_prediction(simTime,r_veh,r_tank,l_tank,m_veh_structure,
    m_prop,F_T_max,epsilon,P0_eng,T0_eng,M_bar_prop,gamma_prop,
    inj_percentDrop,catBed_pressDrop,ox_lineDiameter,f_lineDiameter, OF);



# Since this cell calls the functions defined above, this cell will have all output of other cells above.

Academic license - for non-commercial use only

Oxygen Tank
obj_val max tank pressure: 730.0045862074338
dual m_dot_ox: -21.719609524383966
mean ox_tankPressure 365.0022931037169

Fuel Tank
obj_val max tank pressure: 732.7141674678437
dual m_dot_f: -64.53984652858551
mean f_tankPressure 366.35708373392185

Academic license - for non-commercial use only


# ----------------------------------start here for code tracing------------------------------------  
This cell is what runs all of the above functions and currently outputs any printed optimization results

#### Sensitivity Analysis
This sensitivity analysis was performed as an example of future analysis that can be performed as seen fit. In the future, this code will likely be modified multiple times to analyze the sensitivity of specific parameters individually. This analysis serves as a demonstration of the capabilities of the language, not as something that needs to be examined carefully. 

In this model, we analyze how a change in the mass flow of the propellant changes the operating pressure of the tanks. This might be relevant because tanks must be choosen with appropriate maximum pressure ratings. Tanks with higher pressure ratings might be more expensive or able to contain less fuel. Both of these are prohibitive to the end goal of altitude maximization. The numbers that are computed make sense relative to the parameters chosen. Mass flow only accounts for a portion of the total pressure on the tanks, so a large change in mass flow should not have a very large change on the overall tank pressure. Since the mass flow of the fuel is smaller than the oxidizer, the change in pressure corresponding to the same change in flow has a larger overall effect on the tank pressure. These results make analytic sense, showing that the model is behaving correctly. Whether the results are useful or not is a different question.

Since mass flows are in the single digit kg/s range, the dual variable outputs are large perturbations in the mass flow, on the order of 1 kg/s differences, thus causing large pressure swings shown by dual m_dot_ox and dual m_dot_f. 

Adding more constraints and variables will increase the complexity of the model, and would allow for additional sensitivity analysis.

# Altitude Prediction

From here on, the code is not used for optimization. It takes all of the calculations performed above and uses rocket equations to give altitude vs time and velocity vs time data. This data is then graphed below.

eom_rocket
2nd order non-linear differential equation for vertical ascent and constant thrust

In [106]:
function eom_rocket(y,p,t)
    
    # println(t,", ",y) 
    
    # uncomment this line and check what number 
    # t approaches and choose something with an altitude that is close to zero. It is necessary
    # to choose a simulation time that ends before altitude approaches negative infinity.
    # This is only due to the method used to solve the Ordinary Differential Equation.
    
    
    massFlow = m_dot;
    Pe = Pe_eng;
    
    # altitude, temp and pressure as a function of altitude
    rhoArray = Get_rho_T_P(y[1])
    rho = rhoArray[1];
    T   = rhoArray[2];
    P   = rhoArray[3];
    
    # Get Cd, Ma and Re
    flowArray =  GetFlow(T,rho,y[2])
    Cd = flowArray[1];
    Ma = flowArray[2];
    Re = flowArray[3];

    
    
    # Force due to drag [N]
    F_d = 0.5* Cd * A_veh * rho * y[2]^2;
    
    # Thrust is constant piecwise
    if t < eng_time

        # thrust
        F_T = massFlow*ue+(Pe-P)*Ae;

        # rocket mass decreases linearly with m_dot (assumed ~constant m_dot)
        m_veh = m_veh_wet-massFlow*t;

    else
        F_T = 0;                # Thrust = 0 after motor burnout
        m_veh = m_veh_empty;    # Mass is constant after burnout
        massFlow = 0;
    end

    # returned ydot values
    ret = [y[2],F_T/m_veh-F_d/m_veh-massFlow.*y[2]/m_veh-g]
    
    return ret
end
;

Backout performance
Uses generated eom_rocket ydot array to compute relevant parameters for output

In [107]:
function BackoutPerformance(t,y1,y2)
    # 2nd order non-linear differential equation for vertical ascent and
    # constant thrust

    massFlow = m_dot;
    Pe = Pe_eng;

    y=[y1,y2];      # y1 is altitude, y2 is velocity

    # altitude, temp and pressure as a function of altitude
    rhoArray = Get_rho_T_P(y1)
    rho = rhoArray[1];
    T = rhoArray[2];
    P = rhoArray[3];

    # Get Cd, Ma and Re
    flowArray = GetFlow(T,rho,y[2])
    Cd = flowArray[1];
    Ma = flowArray[2];
    Re = flowArray[3];

    # Force due to drag [N]
    F_d = 0.5* Cd * A_veh * rho * y[2]^2;

    # Thrust is constant piecwise
    if t < eng_time
    
        # thrust
        F_T = massFlow*ue+(Pe-P)*Ae;
    
        
        # rocket mass decreases linearly with m_dot (assumed ~constant m_dot)
        m_veh = m_veh_wet-massFlow*t;
    
    else
        F_T = 0;                # Thrust = 0 after motor burnout
        m_veh = m_veh_empty;    # Mass is constant after burnout
        massFlow = 0;
    end

    # should it be the top one? mirrors the cell above.
    acc = F_T/m_veh-F_d/m_veh-massFlow.*y[2]/m_veh-g
    # returned acceleration
    #acc = F_T/m_veh-F_d/m_veh-g;
    
    return acc, F_T, F_d, Ma, Re, rho, Cd

end
;

Solving EOM

In [108]:
x0 = 0.0;
v0 = 0.0;
IC = [x0, v0];

simTime = 90 # CHANGE ME IF TOO LOW

# Feel free to mess with simTime to prevent error messages. The solver just keeps going negative.
# Basically the solver never quite reaches the simTime because the different iteration values
# have very large changes in yout[1][:] (altitude) for very small time steps
# so the solver tries to compensate by taking even smaller time steps. This is obvious
# if you print t for each iteration in eom_rocket() method. If the last line is -Inf, you need to shorten the simTime
# There is probably a solution for this in the DifferentialEquations package, but this isnt NASA.


t_span = (0.0, simTime) # used in Differential Equations problem statement

# Problem statement
prob = ODEProblem(eom_rocket,IC,t_span);

# Problem Solver using Eulers method
yout = DifferentialEquations.solve(prob,Euler(),dt = 0.1);


# only need data while rocket altitude is greater than or equal to zero
alt =  [] # array for storing altitude of rocket
vel =  [] # array for storing y velocity of rocket
time = [] # time at each index of alt and vel arrays

#For graphing purposes collect data until rocket crashes into ground
altGraph =  [] # array for storing altitude of rocket
velGraph =  [] # array for storing y velocity of rocket
timeGraph = [] # time at each index of alt and vel arrays
for i = 1:length(yout) # all altitude and velocity measurements from diffeq solver
    if(yout[2,i] >= 0) # before apex of trajectory 
        # keep track
        append!(alt,  yout[i][1]);
        append!(vel,  yout[i][2]);
        append!(time, yout.t[i]);
        append!(altGraph, yout[i][1]);
        append!(velGraph, yout[i][2]);
        append!(timeGraph, yout.t[i]);
        
        elseif(yout[1,i] >= 0) # after apex of trajectory
        append!(altGraph, yout[i][1]);
        append!(velGraph, yout[i][2]);
        append!(timeGraph, yout.t[i]);
    else
        # dont keep track
        break;
    end
end

# Back Solving for accleration, Force of Drag, Cd, Ma, Re

# storage arrays
acc_num = zeros(length(time),1);
F_d =     zeros(length(time),1);
F_T =     zeros(length(time),1);
Ma =      zeros(length(time),1);
Re =      zeros(length(time),1);
rho =     zeros(length(time),1);
Cd =      zeros(length(time),1);

# back out acceleration, drag, Mach and Reynolds
for i = 1:length(time)
    # solves for important rocket statistics
    arr = BackoutPerformance(time[i],alt[i],vel[i])
    Cd[i] =  arr[1];
    F_T[i] = arr[2];
    F_d[i] = arr[3];
    Ma[i] =  arr[4];
    Re[i] =  arr[5];
    rho[i] = arr[6];
    Cd[i] =  arr[7];
end
;

# Results and discussion

In [110]:
###### Results output #######

# max drag force
max_Fd = F_d[argmax(F_d)];

# max dynamic pressure
q = zeros(length(time),1);
for i = 1:length(vel)
   q[i] = 1/2*rho[i]*vel[i]^2;
end
maxQ = q[argmax(q)];

# max velocity
maxVelocity = vel[argmax(vel)];

# max Mach #
maxMach = Ma[argmax(Ma)];

# specific impulse
Isp = ue/g;

# whats the payload mass? 10 lbs seems reasonable but I have no idea
m_payload = 10 # Payload Mass [lbs]
m_empty = m_veh_empty - m_payload

println("-------------------------- Rocket Ratios --------------------------")
println("η = ", (m_empty + m_prop + m_payload )/(m_empty + m_payload) )
println("ε = ", m_empty/(m_empty + m_payload) )
println("λ = ", m_payload/(m_empty + m_payload) )
println("")

println("------------------------ Rocket Parameters ------------------------")
println("Vehicle dry weight: ",(m_veh_empty)*kg2lbm," [lb]")
println("Vehicle wet weight: ",(m_veh_wet)*kg2lbm," [lb]")
println("Vehicle diameter: ",2*r_veh*m2ft," [ft]")
println("Propellant mass (total): ",m_prop," [kg], ",m_prop*kg2lbm,", [lb]")
println("Ox:fuel mass ratio: ",OF,"[-]")
println("Oxidizer mass: ",m_ox," [kg], ",m_ox*kg2lbm," [lb]")
println("Fuel mass: ",m_f," [kg], ",m_f*kg2lbm," [lb]")
println("Propellant mass fraction: ",m_prop/m_veh_wet," [-]")
println("Propellant tank Weight : ",tankWt," [lb]")
println("Oxidizer tank Wall thickness : ",ox_t_cyl," [in]")
println("Kerosene tank Wall thickness : ", f_t_cyl," [in]")
println("Oxidizer tank length (inner dim): ",ox_tank_length," [ft]   ")
println("Kerosene tank length (inner dim): ",f_tank_length ," [ft]   ")
println("Oxidizer tank volume: ",vol_ox ," [m^3]   ")
println("Kerosene tank volume: ", vol_f ," [m^3]   ")
println("Oxidizer tank max operating pressure: ", ox_tankPressure*Pa2psi ," [psi]")
println("Kerosene tank max operating pressure: ", f_tankPressure*Pa2psi," [psi]")
println("Oxidizer tank mean operating pressure: ", ox_tankPressure*Pa2psi/2," [psi]" )
println("Kerosene tank mean operating pressure: ",f_tankPressure*Pa2psi/2 ," [psi]")
println("")

## Engine Paramaters
println("----------------------- Propulsion Parameters ---------------------")
println("Sea level thrust: ",F_T[1]/1000 ," [kN], ", F_T[1]/1000*kN2lbf," [lbf]")
println("Expansion ratio: ", expansionRatio ," [-]")
println("Chamber Pressure: ",P0_eng*1e-6 ," [MPa], ",P0_eng*Pa2psi ," [psia]")
println("Exit Pressure: ",Pe_eng*1e-6 ," [MPa], ",Pe_eng*Pa2psi ," [psia]")
println("Chamber temperature ", T0_eng ," [K]")
println("Molar mass of propellant: ",M_bar_prop ," [kg/kmol]")
println("Specific heat ratio of propellant (frozen-flow) ",gamma_prop ," [-]")
println("Pressure at injector: ",(P0_eng+inj_percentDrop*P0_eng/100)*Pa2psi ," [psia]")
println("Mass flow rate (total): ", m_dot ," [kg/s]")
println("Mass flow rate (oxidizer): ", m_dot_ox ," [kg/s], ",m_dot_ox*kg2lbm ," [lb/s]")
println("Mass flow rate (fuel): ",m_dot_f ," [kg/s], ",m_dot_f*kg2lbm ," [lb/s]")
println("Oxidizer feed line diameter ",ox_lineDiameter ," [in]")
println("Fuel feed line diameter ",f_lineDiameter ," [in]")
println("Burn time ",eng_time ," [s]")
println("")

println("----------------------------- Results -----------------------------")
println("Max Altitude: ",alt[end] ," [m], ",3.28*alt[end] ," [ft]")
println("Max Altitude Time: ",round(time[end], digits = 1) ," [s]")
println("Max Velocity: ",maxVelocity ," [m/s]" )
println("Max Mach #: ", maxMach ," [-]")
println("Max Thrust: ",F_T[argmax(F_T)]/1000 ," [kN], ", F_T[argmax(F_T)]/1000*kN2lbf," [lbf]")
println("Specific Impulse: ",Isp ," [s]")
println("Exhaust Velocity: ",ue ," [m/s]")
println("ΔV: ", ue*log(m_veh_wet/m_prop), " [m/s]")
println("Max Drag Force: ",max_Fd/1000 ," [kN], ",max_Fd/1000*kN2lbf ," [lbf]")
println("Max dynamic pressure: ",maxQ/1000 ," [kPa]")
println("Total Impulse: ",F_T[argmax(F_T)]/4.448*eng_time ," [lb-s]") # 1lb = 4.448 Newtons
println("")


-------------------------- Rocket Ratios --------------------------
η = 1.440909397131659
ε = 0.7669731800761169
λ = 0.23302681992388308

------------------------ Rocket Parameters ------------------------
Vehicle dry weight: 94.6079940806868 [lb]
Vehicle wet weight: 136.321547714638 [lb]
Vehicle diameter: 0.666666688 [ft]
Propellant mass (total): 18.920972155723522 [kg], 41.713553633951186, [lb]
Ox:fuel mass ratio: 2.56[-]
Oxidizer mass: 13.606092336700062 [kg], 29.996263287335687 [lb]
Fuel mass: 5.31487981902346 [kg], 11.7172903466155 [lb]
Propellant mass fraction: 0.3059938383421981 [-]
Propellant tank Weight : 6.42320695895983 [lb]
Oxidizer tank Wall thickness : 0.07300045862074338 [in]
Kerosene tank Wall thickness : 0.07327141674678438 [in]
Oxidizer tank length (inner dim): 2.551537814734738 [ft]   
Kerosene tank length (inner dim): 1.4484621852652624 [ft]   
Oxidizer tank volume: 0.012646153306810182 [m^3]   
Kerosene tank volume: 0.006805638792651992 [m^3]   
Oxidizer tank max o

The numbers generated by this code are very similar to the original output estimated by Brandon, confirming that none of the translation is broken. There are some functions that were omitted as they were not necessary for the original purpose of the code. This could be added, but I haven't looked at it yet.

# Conclusion

While these results were great for all of the constraints that are considered, the actual tank dimensions have many other considerations that must be given thought before purchasing. These considerations may or may not be possible to add to this code, and the limitations may need to be estimated within a certain tolerance. This tolerance could be added to this code. This code will likely be another tool in the toolbox, and not something that will give an absolute final result to anything.